In [4]:
#Libraries
import csv
import json
import nltk
import string
import pickle
import requests
import numpy as np
import pandas as pd
from nltk import pos_tag
from sklearn import metrics
from bs4 import BeautifulSoup
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [6]:
# Load data csv format
dataset = pd.read_csv(r'C:\Users\Hp\Desktop\Chatbot\data\data1.csv', sep=';', header=0) 
question = dataset['question']
intent = dataset['post_tag']
#dataset.sort_values(by=['post_tag'], inplace=True)
print(dataset)

                                               question    post_tag
0            Removing Double Or More Slashes and “str=”   .htaccess
1            Protected directories and IP white listing   .htaccess
2     I converted my htaccess to nginx, can someone ...   .htaccess
3     301 Redirect root without cascading new root d...   .htaccess
4     Masking url to show content from different domain   .htaccess
...                                                 ...         ...
9993  Download highest quality audio from YouTube us...  youtube-dl
9994  Using “yum” to update RPM on Red Hat Enterpris...         yum
9995  ZFS data corruption on multiple disks at same ...         zfs
9996      Enable map dragging and zooming in Openlayers        zoom
9997  How to stop zsh from reprinting the previous c...         zsh

[9998 rows x 2 columns]


In [19]:
#Langages
python = ['python', 'numpy', 'dask', 'python-3.x', 'pandas', 'pandas-profiling', 'python-2.7', 'python-3.6', 'python-3.7',
          'python-attrs', 'python-sphinx', 'flask']

javascript = ['javascript', 'jquery', 'jquery-chosen', 'jquery-select2', 'jquery-ui', 'ajax', 'highcharts', 'deno',
              'leaflet', 'webpack', 'node.js', 'vue.js', 'handlebars.js', 'three.js', 'nuxt.js', 'vuetify.js', 
              'discord.js', 'd3.js', 'pixi.js', 'next.js', 'chart.js', 'vuejs2']

java = ['java', 'multithreading', 'jsf', 'jsf-2.2', 'javafx', 'graphql-java', 'java-8', 'javac', 'hibernate', 'gradle', 
        'hadoop']

csharp = ['c#', 'c#-4.0']

android = ['android', 'android-studio', 'android-emulator', 'android-recyclerview', 'android-room', 'xamarin.android']

php = ['php', 'composer-php', 'cakephp-2.0', 'phpmailer', 'symfony4', 'symfony', 'codeigniter', 'laravel', 
       'backpack-for-laravel', 'laravel-5', 'laravel-5.8', 'laravel-7', 'laravel-echo']

r = ['r']

cplusplus = ['c++', 'c++11']

react = ['react-native', 'reactjs', 'react-admin', 'create-react-app', 'react-google-maps', 'react-hooks', 
         'react-leaflet', 'react-redux']

html = ['html', 'h2', 'amp-html', 'html5-audio']

sql = ['hive', 'sql', 'mysql', 'firebase', 'mongodb', 'postgresql', 'sql-server', 'database', 'oracle-sqldeveloper',
       'plsql', 'cassandra', 'azure-sql-database', 'plsqldeveloper', 'ksqldb', 'flask-sqlalchemy', 'sql-server-2016',
       'sqldatatypes', 'sqlite', 'sqlplus', 'tsql', 'graphql', 'db2', 'entity-framework', 'entity-framework-core']

angular = ['angular', 'angular-material', 'angular7', 'angularjs', 'angular8', 'ag-grid-angular']
c = ['c', 'objective-c']
type_script = ['typescript']
scala = ['scala']
go = ['go']
dot_net = ['.net', 'vb.net', '.net-core', 'asp.net', 'asp.net-core', 'asp.net-mvc', 'asp.net-ajax', 'asp.net-core-signalr',
           '.net-standard', 'linq', 'xamarin']
css = ['css', 'postcss'] 
kotlin = ['kotlin', 'kotlin-coroutines']
go = ['go']
matlab = ['matlab']
uml = ['uml']
rust = ['rust']
lua = ['lua'] 
ruby = ['ruby']
awk = ['awk']
haskell = ['haskell']

#framework
framework = ['swift', 'spring', 'spring-boot', 'spring-batch', 'spring-data-jpa', 'spring-cloud-dataflow',
             'spring-data-r2dbc', 'spring-kafka', 'selenium', 'selenium-webdriver', 'selenium-chromedriver',
             'ruby-on-rails', 'ruby-on-rails-5', 'ionic-framework', 'ionic5'] 

#platforms
platform = ['mapbox', 'cordova', 'heroku', 'autodesk-forge', 'cross-platform', 'parse-platform', 'woocommerce', 'magento']

#data
data = ['data-structures', 'data-mining', 'databricks', 'dataframe', 'dataset', 'data-science', 'database-design', 
        'image', 'file', 'pdf', 'datatable','json', 'geojson', 'datatables', 'dataweave', 'datetime', 'arrays', 'ggplot2',
        'snowflake-cloud-data-platform', 'star-schema-datawarehouse', 'string', 'audio', 'powerbi', 'neo4j', 'forms']

#API
api = ['api', 'uber-api', 'wordpress-rest-api', 'qt', 'tradingview-api', 'here-api', 'facebook-graph-api', 'winapi',
       'github-api', 'rest', 'asp.net-web-api', 'google-calendar-api', 'google-api', 'google-drive-api', 
       'azure-api-management']

#system
system = ['git', 'gitignore', 'gitlab', 'github', 'bash', 'terminal', 'powershell', 'wershell-2.0', 'shell',
           'powershell-3.0', 'command-line', 'batch-file', 'cron', 'wordpress', 'wordpress-theming', 'kubernetes', 
           'kubernetes-ingress'] 

#Operating_system
Operating_system = ['ios', 'linux', 'windows', 'oracle', 'raspbian', 'ubuntu-18.04', 'ubuntu-20.04', 'linux-kernel',
                    'microsoft-graph', 'kali-linux', 'ubuntu', 'unix', 'centos', 'macos', 'oracle-apex', 
                    'oracle11g', 'oracle12c', 'oracledb']

#server
server = ['tomcat', 'tomcat9', 'serve', 'apache', 'apache-spark', 'apache-kafka', 'apache-flink', 'apache-zookeeper',
          'apache-camel', 'apache-superset', 'server','apache-mina', 'apache-poi', 'jenkins', 'jenkins-pipeline',
          'aws-billing', 'aws-lambda', 'redis', '.htaccess', 'blazor']


#Google
google = ['google-chrome', 'google-analytics', 'google-data-studio', 'google-app-engine', 'google-bigquery', 
          'google-apps-script', 'google-cloud-platform', 'google-sheets', 'google-adwords', 'google-bi-engine',
          'google-chrome-extension', 'google-cloud-functions', 'google-cloud-run', 'google-cloud-sql',
          'google-cloud-storage', 'google-colaboratory', 'google-compute-engine', 'google-fit', 'google-maps', 
          'google-people', 'google-sheets-formula', 'google-smart-home']

            
#Artificial_Intelligence
ai = ['deep-learning', 'kerberos', 'language-detection', 'logistic-regression', 'linear-programming','random-forest',
      'machine-learning', 'reinforcement-learning', 'tensorflow', 'neural-network', 'tensorflow2.0', 'nlp', 
      'image-processing', 'artificial-intelligence', 'keras', 'pyspark','botframework', 'computer-vision']

IOT = ['arduino', 'raspberry-pi', 'aws-iot', 'robotframework', 'mqtt', 'gps']

software_tool = ['regex', 'jmeter', 'dynamics-crm', 'elasticsearch', 'maven', 'ansible', 'delphi', 'gcc', 'websphere',
                 'caching', 'crystal-reports', 'nginx', 'odoo', 'virtualbox', 'cmake', 'internet-explorer', 'docker',
                 'docker-swarm', 'docker-compose']

communication = ['ip', 'ip-address', 'gateway', 'routes', 'serial-port', 'networking', 'wireless', 'airflow','connection',
                 'xml', 'xpath', 'xslt']

Security = ['logging', 'authentication', 'encryption', 'security', 'cryptography'] 

Mathematics = ['geometry', 'graph', 'graphics', 'hex', 'matplotlib', 'matrix', 'variables', 'statistics']

protocols = ['http-status-code-404', 'openssl', 'nearprotocol', 'protocol-buffers', 'protocols', 'ssl', 'http',
             'oauth-2.0', 'ssh'] 

algorithm = ['algorithm', 'for-loop', 'if-statement', 'loops', 'function', 'list', 'dictionary', 
             'compiler-optimization', 'environment-variables', 'events', 'listview']

Quality = ['validation', 'performance','optimization', 'automation', 'debugging', 'accelerometer'] 



Editor = ['install4j', 'eclipse', 'visual-studio-code', 'sublimetext3', 'jupyter-notebook', 'notepad++', 
          'visual-studio', 'intellij-idea', 'latex', 'salesforce', 'visual-studio-2010', 'visual-studio-2019',
          'markdown']

communication_App = ['ffmpeg', 'youtube', 'facebook', 'gmail', 'email', 'whatsapp',  'notifications', 'youtube-dl',
                     'facebook-login', 'chatbot', 'web-applications', 'zoom']

cloud = ['cloud', 'amazon-cloudformation', 'amazon-cloudwatch', 'firebase-cloud-messaging', 'ibm-cloud', 'soundcloud'] 

amazon = ['amazon-dynamodb', 'amazon-ec2', 'amazon',  'amazon-cognito', 'amazon-ecs', 'amazon-lex', 'amazon-s3',
          'amazon-ses'] 

In [20]:
labels = {'python', 'javascript', 'java', 'csharp', 'android', 'php', 'r', 'cplusplus', 'react', 'html', 'sql', 'Operating_system', 'angular'}
dataset["label"] = ''

for j in range(0, 9997):
    if dataset['post_tag'][j] in python:
        dataset["label"][j] = 'python' 
    elif dataset['post_tag'][j] in javascript:
        dataset["label"][j] = 'javascript'
    elif dataset['post_tag'][j] in java:
        dataset["label"][j] = 'java'
    elif dataset['post_tag'][j] in csharp:
        dataset["label"][j] = 'csharp'
    elif dataset['post_tag'][j] in android:
        dataset["label"][j] = 'android'
    elif dataset['post_tag'][j] in php:
        dataset["label"][j] = 'php'
    elif dataset['post_tag'][j] in r:
        dataset["label"][j] = 'r'
    elif dataset['post_tag'][j] in cplusplus:
        dataset["label"][j] = 'cplusplus'
    elif dataset['post_tag'][j] in react:
        dataset["label"][j] = 'react'
    elif dataset['post_tag'][j] in html:
        dataset["label"][j] = 'html'
    elif dataset['post_tag'][j] in sql:
        dataset["label"][j] = 'sql'
    elif dataset['post_tag'][j] in Operating_system:
        dataset["label"][j] = 'Operating_system'
    elif dataset['post_tag'][j] in angular:
        dataset["label"][j] = 'angular'
    elif dataset['post_tag'][j] in c:
        dataset["label"][j] = 'c'
    elif dataset['post_tag'][j] in type_script:
        dataset["label"][j] = 'type_script'
    elif dataset['post_tag'][j] in scala:
        dataset["label"][j] = 'scala'
    elif dataset['post_tag'][j] in go:
        dataset["label"][j] = 'go'
    elif dataset['post_tag'][j] in dot_net:
        dataset["label"][j] = 'dot_net'
    elif dataset['post_tag'][j] in ai:
        dataset["label"][j] = 'ai'
    elif dataset['post_tag'][j] in software_tool:
        dataset["label"][j] = 'software_tool'
    elif dataset['post_tag'][j] in communication:
        dataset["label"][j] = 'communication'
    elif dataset['post_tag'][j] in Security:
        dataset["label"][j] = 'Security'
    elif dataset['post_tag'][j] in Mathematics:
        dataset["label"][j] = 'Mathematics'
    elif dataset['post_tag'][j] in IOT:
        dataset["label"][j] = 'IOT'
    elif dataset['post_tag'][j] in protocols:
        dataset["label"][j] = 'protocols'
    elif dataset['post_tag'][j] in algorithm:
        dataset["label"][j] = 'algorithm'
    elif dataset['post_tag'][j] in Quality:
        dataset["label"][j] = 'Quality'
    elif dataset['post_tag'][j] in Editor:
        dataset["label"][j] = 'Editor'
    elif dataset['post_tag'][j] in communication_App:
        dataset["label"][j] = 'communication_App'
    elif dataset['post_tag'][j] in cloud:
        dataset["label"][j] = 'cloud'
    elif dataset['post_tag'][j] in amazon:
        dataset["label"][j] = 'amazon'
    elif dataset['post_tag'][j] in server:
        dataset["label"][j] = 'server'
    elif dataset['post_tag'][j] in google:
        dataset["label"][j] = 'google'
    elif dataset['post_tag'][j] in platform:
        dataset["label"][j] = 'platform'
    elif dataset['post_tag'][j] in data:
        dataset["label"][j] = 'data'
    elif dataset['post_tag'][j] in system:
        dataset["label"][j] = 'system'
    elif dataset['post_tag'][j] in framework:
        dataset["label"][j] = 'framework'
    elif dataset['post_tag'][j] in api:
        dataset["label"][j] = 'api'
    elif dataset['post_tag'][j] in css:
        dataset["label"][j] = 'css'
    elif dataset['post_tag'][j] in kotlin:
        dataset["label"][j] = 'kotlin'
    elif dataset['post_tag'][j] in matlab:
        dataset["label"][j] = 'matlab'
    elif dataset['post_tag'][j] in uml:
        dataset["label"][j] = 'uml'
    elif dataset['post_tag'][j] in rust:
        dataset["label"][j] = 'rust'
    elif dataset['post_tag'][j] in lua:
        dataset["label"][j] = 'lua'
    elif dataset['post_tag'][j] in ruby:
        dataset["label"][j] = 'ruby'
    elif dataset['post_tag'][j] in haskell:
        dataset["label"][j] = 'haskell'
    else:
        dataset['label'][j] = 'no_meaning'
       

In [21]:
import collections
counter=collections.Counter(dataset['label'])
print(counter)

Counter({'python': 1619, 'javascript': 1334, 'no_meaning': 1255, 'java': 687, 'sql': 485, 'csharp': 424, 'php': 402, 'android': 359, 'Operating_system': 330, 'r': 271, 'cplusplus': 268, 'react': 264, 'system': 221, 'software_tool': 208, 'html': 204, 'framework': 202, 'angular': 162, 'data': 155, 'c': 130, 'dot_net': 115, 'server': 113, 'google': 82, 'ai': 77, 'algorithm': 62, 'Editor': 60, 'css': 59, 'api': 56, 'type_script': 44, 'communication': 38, 'communication_App': 37, 'scala': 35, 'platform': 31, 'Quality': 30, 'protocols': 28, 'IOT': 20, 'go': 20, 'matlab': 20, 'amazon': 14, 'Security': 14, 'kotlin': 12, 'rust': 10, 'lua': 9, 'ruby': 9, 'Mathematics': 8, 'cloud': 7, 'haskell': 6, 'uml': 1, '': 1})


In [22]:
def Pos_tagging (pos_tag):
     if pos_tag.startswith('J'):
         return wordnet.ADJ
     elif pos_tag.startswith('V'):
         return wordnet.VERB
     elif pos_tag.startswith('N'):
         return wordnet.NOUN
     elif pos_tag.startswith('R'):
         return wordnet.ADV
     else:
         return wordnet.NOUN

In [23]:
def cleaning (question):
        question = question.lower()
        question = [word.strip(string.punctuation) for word in question.split()]
        question = [word for word in question if not any(c.isdigit() for c in word)]
        stop_words = stopwords.words('english')
        question = [x for x in question if x not in stop_words]
        unhelpful_info = {'where','what', 'why', 'how', 'error', 'when' }
        question = [x for x in question if x not in unhelpful_info]
        question = [t for t in question if len(t) > 0]
        pos_tags = pos_tag(question)
        question = [WordNetLemmatizer().lemmatize(t[0],Pos_tagging(t[1])) for t in pos_tags]
        question = [t for t in question if len(t) > 1]
        question = " ".join(question)
        return(question)

In [24]:
# apply Cleaning function to questions
dataset["clean"] = dataset['question'].apply(lambda x:cleaning(x))

In [25]:
# Labels are the values we want to predict
labels = dataset["label"]
features = dataset["clean"]

In [26]:
# One-hot encode the data using pandas get_dummies
#labels = pd.get_dummies(labels)

In [27]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.3, random_state = 42)

In [28]:
#    ______ Building model "Random Forest Model" ______

# Pipeline for transforming data, fitting to model and predicting
model_pipeline = Pipeline(steps=[
        ('vect', CountVectorizer(stop_words='english')),
        ('tfidf', TfidfTransformer()), 
        ('clf', RandomForestClassifier())])
# Instantiate model with 100 decision trees  
parameters = {
        'clf__min_samples_split': [10],
        'clf__n_estimators':      [100]
       #'clf__max_leaf_nodes':    [25],
       #'clf__max_depth':         [20],
       #'clf__min_samples_leaf':  [5]
}

#GridSearch with the above parameters
model = GridSearchCV(
        model_pipeline, param_grid=parameters, scoring='accuracy', verbose=2
   )

In [29]:
# Training model "Random Forest Model"
model.fit(X_train, y_train) 

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] clf__min_samples_split=10, clf__n_estimators=100 ................


C:\Users\Hp\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . clf__min_samples_split=10, clf__n_estimators=100, total=   4.6s
[CV] clf__min_samples_split=10, clf__n_estimators=100 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.5s remaining:    0.0s


[CV] . clf__min_samples_split=10, clf__n_estimators=100, total=   5.0s
[CV] clf__min_samples_split=10, clf__n_estimators=100 ................
[CV] . clf__min_samples_split=10, clf__n_estimators=100, total=   4.9s
[CV] clf__min_samples_split=10, clf__n_estimators=100 ................
[CV] . clf__min_samples_split=10, clf__n_estimators=100, total=   5.9s
[CV] clf__min_samples_split=10, clf__n_estimators=100 ................
[CV] . clf__min_samples_split=10, clf__n_estimators=100, total=   4.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   25.4s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [30]:
#    ______ Evaluating model ______

y_pred = model.predict(X_test)

print("Test accuracy:\n",accuracy_score(y_test, y_pred)*100)


Test accuracy:
 45.53333333333333


In [34]:
model_filepath = 'C:/Users/Hp/Desktop/Precious_Model/Random_Forest_Model.pkl'
def save_model(model: GridSearchCV, model_filepath: str) -> None:
    """
    save the model to the designated model_filepath
    :param model: GridSearchCV model to be saved
    :param model_filepath: the file path for the saved model
    :return: None
    """

    with open(model_filepath, 'wb') as file:
        pickle.dump(model, file)

In [35]:
print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)
print('Trained model saved!')

Saving model...
    MODEL: C:/Users/Hp/Desktop/Precious_Model/Random_Forest_Model.pkl
Trained model saved!
